<a href="https://colab.research.google.com/github/mmtos/BertStudy/blob/master/RunSquad_ipynb(20190310).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#tensorflow2.0 버전과 연동하기 어려움
#실행하지 말것. 실행했다면 런타임 재설정.
!pip install tensorflow==2.0.0-alpha0 
print(tf.__version__) 

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
#from tensorflow.contrib import tpu

In [0]:

#구글 계정 인증
from google.colab import auth
auth.authenticate_user()


In [47]:
# CLOUD TPU CLUSTER 사용하기 (2.0 버전에 Session 클래스 없어짐)
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
#TPU_ADDRESS = 'grpc://34.80.57.254:8470'
#TPU_ADDRESS = 'grpc://10.240.1.10:8470'

print('TPU address is', TPU_ADDRESS)
with tf.Session(TPU_ADDRESS) as session:
  #session.run(tpu.initialize_system())
  #session.run(tf.global_variables_initializer())
  
  print('TPU devices:')
  pprint.pprint(session.list_devices())
  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)



TPU address is grpc://10.111.248.154:8470


In [0]:
#bert_repo 다운로드(colab 좌측 화살표 클릭후 ..> content > bert_repo 폴더에서 완료여부 확인가능)
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo

In [0]:
#명령 path 설정(안해줘도 될것같음)
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

In [0]:
# Now credentials are set for all future sessions on this TPU.
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Run BERT on SQuAD 1.1 and SQuAD 2.0."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import json
import math
import os
import random
import modeling
import optimization
import tokenization
import six
import tensorflow as tf



In [0]:
#squad데이터 다운로드
!mkdir squad_data

In [0]:
#squad데이터 다운로드
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!wget https://raw.githubusercontent.com/allenai/bi-att-flow/master/squad/evaluate-v1.1.py
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json


--2019-03-10 07:50:30--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json’

dev-v1.1.json       100%[===================>]   4.63M  --.-KB/s    in 0.1s    

2019-03-10 07:50:31 (35.0 MB/s) - ‘dev-v1.1.json’ saved [4854279/4854279]

--2019-03-10 07:50:32--  https://raw.githubusercontent.com/allenai/bi-att-flow/master/squad/evaluate-v1.1.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3419 (3.3K) [text/plain]
Saving to: ‘evaluate-v1.1.p

In [0]:
#squad데이터 준비완료
!mv dev-v1.1.json squad_data
!mv train-v1.1.json squad_data
!mv evaluate-v1.1.py squad_data
!ls squad_data

dev-v1.1.json  evaluate-v1.1.py  train-v1.1.json


In [52]:
#runsquad.py 수정없이 사용하기위해
#내부 스크립트와 사전연동작업 수행

#데이터 폴더 지정(TASK는 다른 예제에서도 재사용될것으로 예상됨)
TASK = 'bert_base_squad' #@param {type:"string"}
SQUAD_DIR= 'squad_data/'

#구글 클라우드 플랫폼(GCP) 내 GCP Storage 지정
BUCKET = 'mybert' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'

#OUTPUT경로가 로컬에 있을경우 에러가 발생하기때문에
#GCP Storage내에 결과저장경로 설정
OUTPUT_DIR = 'gs://{}/bert/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://mybert/bert/models/bert_base_squad *****


In [53]:
#USING BASE BERT_UNCASE MODEL WITH COLAB SINGLE TPU 
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model

#여러 bert모델 선택(PRETRAINED_DIR 지정할 때 사용됨)
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}

#사전 훈련된 bert모델 경로 지정
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
!gsutil ls $BERT_PRETRAINED_DIR


***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


In [0]:
#!mkdir bert_model
#!gsutil cp -r $BERT_PRETRAINED_DIR bert_model
#!ls bert_model
#!rm -rfv bert_model

In [54]:
#사전학습된 모델 다운로드
MODEL_DIR = 'gs://{}/bert/models/'.format(BUCKET)+BERT_MODEL
tf.gfile.MakeDirs(MODEL_DIR)
!gsutil cp -r $BERT_PRETRAINED_DIR $MODEL_DIR

Copying gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json [Content-Type=application/json]...
Copying gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index [Content-Type=application/octet-stream]...
Copying gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta [Content-Type=application/octet-stream]...
- [4 files][420.9 MiB/420.9 MiB]  554.3 KiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint [Content-Type=application/octet-stream]...
Copying gs://cloud-tpu-checkpoint

In [55]:
BERT_PRETRAINED_DIR=MODEL_DIR+'/'+BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))


gs://mybert/bert/models/uncased_L-12_H-768_A-12/uncased_L-12_H-768_A-12


In [0]:
#run_squad 실행(내부에서 사용될 파라미터도 같이 지정)
#(CPU 사용)
#2018. 12월 버전
!python bert_repo/run_squad.py \
  --vocab_file=$BERT_PRETRAINED_DIR/vocab.txt \
  --bert_config_file=$BERT_PRETRAINED_DIR/bert_config.json \
  --init_checkpoint=$BERT_PRETRAINED_DIR/bert_model.ckpt \
  --do_train=True \
  --train_file=$SQUAD_DIR/train-v1.1.json \
  --do_predict=True \
  --predict_file=$SQUAD_DIR/dev-v1.1.json \
  --train_batch_size=12 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=/tmp/squad_base/

INFO:tensorflow:Using config: {'_model_dir': '/tmp/squad_base/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb57a452780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, in

In [0]:
##################### 시행착오 남김 실행하지말것. ##############################
!python bert_repo/run_squad.py \
  --vocab_file=$BERT_PRETRAINED_DIR/vocab.txt \
  --bert_config_file=$BERT_PRETRAINED_DIR/bert_config.json \
  --init_checkpoint=$BERT_PRETRAINED_DIR/bert_model.ckpt \
  --do_train=True \
  --train_file=$SQUAD_DIR/train-v1.1.json \
  --do_predict=True \
  --predict_file=$SQUAD_DIR/dev-v1.1.json \
  --train_batch_size=16 \
  --learning_rate=3e-3 \
  --num_train_epochs=1.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=/tmp/squad_base_tpu/ \
  --use_tpu=True \
  --tpu_name=$TPU_ADDRESS
#  --tpu_zone=$TPU_ZONE \
#  --gcp_project=$PROJECT_NAME
#  --tpu_name=$TPU_NAME \
#FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': '/tmp/squad_base_tpu/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.111.248.154:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efd67c44dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.11

In [56]:
#run_squad 실행(TPU Cluster 사용, 내부에서 사용될 파라미터도 같이 지정)
#Training 및 Test Set Predict결과 json 형식으로 저장함.
!python bert_repo/run_squad.py \
  --vocab_file=$BERT_PRETRAINED_DIR/vocab.txt \
  --bert_config_file=$BERT_PRETRAINED_DIR/bert_config.json \
  --init_checkpoint=$BERT_PRETRAINED_DIR/bert_model.ckpt \
  --do_train=True \
  --train_file=$SQUAD_DIR/train-v1.1.json \
  --do_predict=True \
  --predict_file=$SQUAD_DIR/dev-v1.1.json \
  --train_batch_size=16 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=$OUTPUT_DIR \
  --use_tpu=True \
  --tpu_name=$TPU_ADDRESS


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': 'gs://mybert/bert/models/bert_base_squad', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.111.248.154:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa5ef55cdd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_mas

In [58]:
#OUTPUT_DIR에 있는 prediction결과를 SQUAD_DIR폴더에 복사함
!gsutil cp $OUTPUT_DIR/predictions.json $SQUAD_DIR
!ls $SQUAD_DIR

Copying gs://mybert/bert/models/bert_base_squad/predictions.json...
/ [1 files][416.3 KiB/416.3 KiB]                                                
Operation completed over 1 objects/416.3 KiB.                                    
dev-v1.1.json  evaluate-v1.1.py  predictions.json  train-v1.1.json


In [59]:
#정확도 스코어 측정(속도를 위해 epoch를 1.0으로 해서 정확도가 낮음)
!python $SQUAD_DIR/evaluate-v1.1.py squad_data/dev-v1.1.json squad_data/predictions.json

{"exact_match": 0.6717123935666982, "f1": 1.9178409403828267}


In [60]:
# ****************TRAIN WITH LARGE MODEL & CLOUD TPU CLUSTER*********

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-24_H-1024_A-16' #@param {type:"string"}
BERT_LARGE_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
MODEL_DIR = 'gs://{}/bert/models/'.format(BUCKET)+BERT_MODEL
tf.gfile.MakeDirs(MODEL_DIR)

Copying gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_config.json [Content-Type=application/json]...
Copying gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index [Content-Type=application/octet-stream]...
Copying gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.meta [Content-Type=application/octet-stream]...
| [4 files][  1.3 GiB/  1.3 GiB]    1.1 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/checkpoint [Content-Type=application/octet-stream]...
Copying gs://cloud-tpu-check

In [0]:
#1GB정도 되므로 클라우드내에 다운받았다면 다시 실행하지 말것.
!gsutil cp -r $BERT_LARGE_DIR $MODEL_DIR

In [61]:
BERT_LARGE_DIR=MODEL_DIR+'/'+BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_LARGE_DIR))

SQUAD_DIR= 'squad_data/'

TASK = "bert_large_squad"
BUCKET = 'mybert' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** BERT pretrained directory: gs://mybert/bert/models/uncased_L-24_H-1024_A-16/uncased_L-24_H-1024_A-16 *****
***** Model output directory: gs://mybert/bert/models/bert_large_squad *****


In [62]:
!python bert_repo/run_squad.py \
  --vocab_file=$BERT_LARGE_DIR/vocab.txt \
  --bert_config_file=$BERT_LARGE_DIR/bert_config.json \
  --init_checkpoint=$BERT_LARGE_DIR/bert_model.ckpt \
  --do_train=True \
  --train_file=$SQUAD_DIR/train-v1.1.json \
  --do_predict=True \
  --predict_file=$SQUAD_DIR/dev-v1.1.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=$OUTPUT_DIR \
  --use_tpu=True \
  --tpu_name=$TPU_ADDRESS


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': 'gs://mybert/bert/models/bert_large_squad', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.111.248.154:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f66691f2dd8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_ma

In [63]:
#OUTPUT_DIR에 있는 prediction결과를 SQUAD_DIR폴더에 복사함
!gsutil cp $OUTPUT_DIR/predictions.json $SQUAD_DIR
!ls $SQUAD_DIR

Copying gs://mybert/bert/models/bert_large_squad/predictions.json...
/ [1 files][575.8 KiB/575.8 KiB]                                                
Operation completed over 1 objects/575.8 KiB.                                    
dev-v1.1.json  evaluate-v1.1.py  predictions.json  train-v1.1.json


In [64]:
#정확도 스코어 측정
!python $SQUAD_DIR/evaluate-v1.1.py squad_data/dev-v1.1.json squad_data/predictions.json

{"exact_match": 83.29233680227058, "f1": 90.27251053160043}
